In [1]:
import pandas as pd

# sample df
df = pd.DataFrame({'id': [1, 2, 3],
                   'name': ['A', 'B', 'C'],
                   'roomno': [115, 115, 117],
                   'date_start': pd.to_datetime(['2023-04-04 08:00:00', '2023-04-04 12:00:00', '2023-04-04 08:00:00']),
                   'date_end': pd.to_datetime(['2023-04-04 14:00:00', '2023-04-04 18:00:00', '2023-04-04 14:00:00'])})

room_cap = pd.DataFrame({'roomno': [115, 117],
                         'cap': [3, 1]})

# reformat to longer
df_long = (df
           .melt(id_vars=['id', 'name', 'roomno'],
                 value_vars=['date_start', 'date_end'], 
                 var_name='what', 
                 value_name='time')
            .groupby('id'))

# df_long = df_long.groupby('id')

# define function to create date range for each group
def create_date_range(group):
    start = group['time'].min()
    end = group['time'].max()
    return pd.date_range(start=start, end=end, freq='H')

# apply function to each group and concatenate results into a single DataFrame
hourly_df = pd.concat([pd.DataFrame({'id': id,
                                     'time': create_date_range(group)}) for id, group in df_long])

# # merge base information back
hourly_df = pd.merge(hourly_df,
                     df[["id", "name", "roomno"]],
                     on='id', how='left')

# add count per hour and calculate occupancy rate
hourly_occ = (hourly_df
             .groupby(['roomno', 'time'])
             .size()
             .reset_index(name='n')
             .merge(room_cap, on='roomno')
             .assign(occ_rate=lambda x: x['n'] / x['cap']))

print(hourly_occ)

    roomno                time  n  cap  occ_rate
0      115 2023-04-04 08:00:00  1    3  0.333333
1      115 2023-04-04 09:00:00  1    3  0.333333
2      115 2023-04-04 10:00:00  1    3  0.333333
3      115 2023-04-04 11:00:00  1    3  0.333333
4      115 2023-04-04 12:00:00  2    3  0.666667
5      115 2023-04-04 13:00:00  2    3  0.666667
6      115 2023-04-04 14:00:00  2    3  0.666667
7      115 2023-04-04 15:00:00  1    3  0.333333
8      115 2023-04-04 16:00:00  1    3  0.333333
9      115 2023-04-04 17:00:00  1    3  0.333333
10     115 2023-04-04 18:00:00  1    3  0.333333
11     117 2023-04-04 08:00:00  1    1  1.000000
12     117 2023-04-04 09:00:00  1    1  1.000000
13     117 2023-04-04 10:00:00  1    1  1.000000
14     117 2023-04-04 11:00:00  1    1  1.000000
15     117 2023-04-04 12:00:00  1    1  1.000000
16     117 2023-04-04 13:00:00  1    1  1.000000
17     117 2023-04-04 14:00:00  1    1  1.000000
